In [14]:
import os.path

!pip install transformers librosa soundfile numpy"<"2 torch torchaudio fast-tsp kmedoids


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)



[notice] A new release of pip is available: 23.2.1 -> 25.1.1
[notice] To update, run: pip install --upgrade pip


# Setup

In [1]:
%load_ext autoreload
%autoreload 2

from audio_orderer import AudioOrderer, CLAPWrapper, SmearModifier

import torch
torch.set_grad_enabled(False)

clap = CLAPWrapper()

objc[41940]: Class AVFFrameReceiver is implemented in both /usr/local/Cellar/ffmpeg/6.1_1/lib/libavdevice.60.3.100.dylib (0x171d0d378) and /Users/damian/2.current/clapSlice/venv/lib/python3.12/site-packages/av/.dylibs/libavdevice.61.3.100.dylib (0x18a7b03b0). One of the two will be used. Which one is undefined.
objc[41940]: Class AVFAudioReceiver is implemented in both /usr/local/Cellar/ffmpeg/6.1_1/lib/libavdevice.60.3.100.dylib (0x171d0d3c8) and /Users/damian/2.current/clapSlice/venv/lib/python3.12/site-packages/av/.dylibs/libavdevice.61.3.100.dylib (0x18a7b0400). One of the two will be used. Which one is undefined.


initilizing CLAP


# Run

In [2]:
filename = '/Users/damian/2.current/clapSlice/outputs/Dua Lipa - Be The One (Official Music Video).mp3'
bpm = 87.5

audio_orderer = AudioOrderer(clap=clap, source_audio_path=filename, bpm=bpm)

loaded waveform with shape torch.Size([2, 9805880]) , sampling rate 48000


In [3]:
chunk_beats = 2
window_width = 0
sort_order = audio_orderer.make_order(chunk_beats=chunk_beats, preserve_start_and_end=True)
sort_order

computing distance matrix
computing route


AudioOrdering(source_audio='/Users/damian/2.current/clapSlice/outputs/Dua Lipa - Be The One (Official Music Video).mp3', bpm=87.5, chunk_beats=2, sort_order=tensor([  0,   1,   2,   3,   4,   6,   5,  38,  95,  39,  94,  93,  37,  22,
         21,  13,  14,  15,  16,   8,   9,  17,  20,  73,  72,  74,  66,  76,
         68,  64,  65,  25,  81,  89,  88,  24,  80,  90,  34,  82,  26,  28,
         84,  33,  32,  79,  99, 131,  43, 143,  55, 101, 133,  45,  47, 135,
        103,  41,  97, 129,  40,  96,  98,  42, 130,  35,  91, 100,  44, 132,
        134,  46, 102,  75,  71,  67,  31,  87,  27,  83,  11, 123, 115, 126,
         19,  36,  92, 112, 117, 113, 114, 118, 121, 125, 120, 124, 116, 119,
        128,  51, 139, 107, 108, 140,  52,  48, 104, 136, 138, 105, 137,  49,
        141, 109,  53,  54, 142, 106,  50, 110,  78,  77,  69,  85,  29,  30,
         86,  70,  60,  56,  57,  62,  61,  58,  59,  63,  18, 122,  12,  10,
         23, 111, 127,   7, 144, 145, 146, 147]), window_width=

In [18]:
sort_order_2 = audio_orderer.make_order(chunk_beats=chunk_beats, preserve_start_and_end=True)
sort_order_2

computing distance matrix
computing route


AudioOrdering(source_audio='/Users/damian/2.current/clapSlice/outputs/Dua Lipa - Be The One (Official Music Video).mp3', bpm=87.5, chunk_beats=2, sort_order=tensor([  0,   1,   2,   3,   4,   6,   5,  38,  95,  39,  94,  93,  37,  22,
         21,  13,  14,  15,  16,   8,   9,  17,  20,  73,  72,  74,  66,  76,
         68,  64,  65,  25,  81,  89,  88,  24,  80,  90,  34,  82,  26,  28,
         84,  33,  32,  79,  99, 131,  43, 143,  55, 101, 133,  45,  47, 135,
        103,  41,  97, 129,  40,  96,  98,  42, 130,  35,  91, 100,  44, 132,
        134,  46, 102,  75,  71,  67,  31,  87,  27,  83,  11, 123, 115, 126,
         19,  36,  92, 112, 117, 113, 114, 118, 121, 125, 120, 124, 116, 119,
        128,  51, 139, 107, 108, 140,  52,  48, 104, 136, 138, 105, 137,  49,
        141, 109,  53,  54, 142, 106,  50, 110,  78,  77,  69,  85,  29,  30,
         86,  70,  60,  56,  57,  62,  61,  58,  59,  63,  18, 122,  12,  10,
         23, 111, 127,   7, 144, 145, 146, 147]), window_width=

In [19]:
sort_order.source_audio = filename
sort_order.bpm = bpm
from audio_ordering_candidates_registry import save_candidate
save_candidate(sort_order)

In [4]:
from audio_orderer import SmearModifier

smear_modifiers = [
    SmearModifier(smear_width=1, spread=4, match_embedding=clap.get_text_features("vocal, song, emotional singing")),
    SmearModifier(smear_width=2, spread=1, match_embedding=clap.get_text_features("instrumental"))
]

In [17]:

from IPython.display import Audio

result = audio_orderer.apply_order(
    audio_ordering=sort_order,
    smear_width=2,
    spread=0,
    wrap_mode='bleed',
    save=True,
    smear_modifiers=smear_modifiers,
    smooth_smear_modifiers=True,
    print_smear_sources=True
)
!open outputs
#Audio(result.output_audio, rate=audio_orderer.sampling_rate)
#print(ordered.shape)

[0, 1, 2, 3, 4, 6, 5, 38, 95, 39, 94, 93, 37, 22, 21, 13, 14, 15, 16, 8, 9, 17, 20, 73, 72, 74, 66, 76, 68, 64, 65, 25, 81, 89, 88, 24, 80, 90, 34, 82, 26, 28, 84, 33, 32, 79, 99, 131, 43, 143, 55, 101, 133, 45, 47, 135, 103, 41, 97, 129, 40, 96, 98, 42, 130, 35, 91, 100, 44, 132, 134, 46, 102, 75, 71, 67, 31, 87, 27, 83, 11, 123, 115, 126, 19, 36, 92, 112, 117, 113, 114, 118, 121, 125, 120, 124, 116, 119, 128, 51, 139, 107, 108, 140, 52, 48, 104, 136, 138, 105, 137, 49, 141, 109, 53, 54, 142, 106, 50, 110, 78, 77, 69, 85, 29, 30, 86, 70, 60, 56, 57, 62, 61, 58, 59, 63, 18, 122, 12, 10, 23, 111, 127, 7, 144, 145, 146, 147]
smear width: 2.085270404815674  spread: 0.7441884279251099 -> 2 1
smear width: 2.086822271347046  spread: 0.7395330667495728 -> 2 1
smear width: 2.2614548206329346  spread: 0.21563565731048584 -> 2 1
smear width: 2.113037586212158  spread: 0.6608871221542358 -> 2 1
smear width: 2.078572988510132  spread: 0.7642813324928284 -> 2 1
smear width: 2.0363149642944336  spre

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


# Video